# Create the Dataset
Use the whisper API to get transcipts and then use ChatGPT to get if its interesting or not.

In [18]:
%pip install pytube
%pip install youtube-dl
%pip install openai
%pip install moviepy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import config

import pytube

import requests
import subprocess
import os
import shutil
import ast

import openai

from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.audio.io.AudioFileClip import AudioFileClip

import youtube_dl

In [20]:
openai.api_key = config.OAI_API_TOKEN

In [22]:
# YouTube video URL
vid_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
fps = 30

# how long the testing clips should be
testing_clips_time = 20

# How high the sentiment has to be for it to be in the video
sentiment_score = 0.6

# debug
delete_directories = False

In [23]:
# Get the audio stream
video_stream = pytube.YouTube(vid_url).streams.filter(only_audio=True).first()
# Check if audio stream is not None before attempting to download it
if video_stream is not None:
    # Download the audio stream
    audio_file = video_stream.download(filename="input.mp4").replace("\\","/")
else:
    print("No audio stream found.")


Break the video down into 25mb chunks

In [24]:
def split_video_by_seconds(input_file_path, chunk_duration):
    """
    Splits a video file into smaller chunks using ffmpeg.
    """
    # Get the duration of the input video
    duration_command = ["ffprobe", "-i", input_file_path, "-show_entries", "format=duration", "-v", "quiet", "-of", "csv=p=0"]
    duration_output = subprocess.check_output(duration_command, encoding="utf-8")
    input_duration = float(duration_output.strip())

    # Calculate the number of chunks and their start and end times
    chunk_start_times = range(0, int(input_duration), int(chunk_duration))
    chunk_end_times = [min(start_time + chunk_duration, input_duration) for start_time in chunk_start_times]

    # Split the video into chunks using ffmpeg
    for i, (start_time, end_time) in enumerate(zip(chunk_start_times, chunk_end_times)):
        output_file_path = f"downloads/input_{i}.mp4"
        split_command = ["ffmpeg", "-i", input_file_path, "-ss", str(start_time), "-to", str(end_time), "-c", "copy", output_file_path]
        subprocess.run(split_command, check=True)
        
def split_file(input_file_path, chunk_size):
    """
    Splits a file into smaller chunks and saves them to disk.
    """
    with open(input_file_path, 'rb') as input_file:
        for chunk_num, chunk in enumerate(iter(lambda: input_file.read(chunk_size), b'')):
            # Save the chunk to disk
            output_file_path = f"downloads/input_{chunk_num}.mp4"
            with open(output_file_path, 'wb') as output_file:
                output_file.write(chunk)

newpath = os.getcwd() + "/downloads/"
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
input_file_path = 'input.mp4'
split_video_by_seconds(input_file_path, testing_clips_time)

Create metadata for the files <br/>
<sub>Start time and duration</sub>

In [25]:
newpath = os.getcwd() + "/metadata/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

start_time = 0.0

for i, filename in enumerate(os.listdir('downloads')):
    md_output_file_path = f"metadata/input_{i}.txt"
    with open(md_output_file_path, 'w') as output_file:
        clip = AudioFileClip('downloads/'+filename)
        duration = clip.duration
        clip.close()
        output_file.write(f"[{start_time},{duration}]")
        start_time += duration

Input the audio into Whisper and get transcription


In [26]:
newpath = os.getcwd() + "/chunks/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

url = 'https://api.openai.com/v1/audio/transcriptions'
headers = {
    'Authorization': f'Bearer {config.OAI_API_TOKEN}'
}
data = {
    'model':'whisper-1',
    'prompt':'Woah! I for a fact don\'t believe that... Okay, I know. I love those~'
}
for chunk_num, filename in enumerate(os.listdir('downloads')):
    with open("downloads/" + filename, "rb") as input_file:
        files = {
            'file':(filename,input_file)
        }
        transcription = requests.post(url=url, headers=headers, files=files, data=data)
        output_file_path = f"chunks/input_{chunk_num}.txt"
        with open(output_file_path, 'w', encoding="utf-8") as output_file:
            output_file.write(transcription.json().get('text'))

### ChatGPT Implementation

In [27]:
url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {config.OAI_API_TOKEN}'
}

Use ChatGPT to break down the text and meta data into workable chunks

In [28]:
for filename in os.listdir('chunks'):
    with open("temporary.txt", "a", encoding="utf-8") as temporary:
        with open("chunks/" + filename, "r", encoding="utf-8") as chunk:
            with open("metadata/" + filename, "r", encoding="utf-8") as timestamp:
                temporary.write(f"[{timestamp.read()},\"{chunk.read()}\"],")

with open("temporary.txt","r", encoding="utf-8") as input_file:
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": f"Using this list of [start_time, duration, video_transcription]. Respond with only a new list of chunks of the text you as an AI model find appropriately fits in speaking english by combinding list elements and then adding the start time and duration together to create the new list of [start_time, duration, chunk]. DO NOT give any answer outisde of the format. \n\"{input_file.read()}\""}] 
    }
    response = requests.post(url=url, headers=headers, json=data)
    created_chunks = response.json().get('choices')[0].get('message').get('content')
    with open("video_transcription.txt", "w", encoding="utf-8") as vidtrans:
        vidtrans.write(created_chunks)

Get sentiment from ChatGPT and create sentiment analysis file

In [29]:
with open("video_transcription.txt", "r", encoding="utf-8") as input_file:
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": f"Using the list [start_time, duration, sentence]. For how 'entertaining a casual viewer of Youtube would find the sentence as part of a video' in any definition to the best of your ability as an AI model the sentence is create a score as a float value to the best of your ability as an AI model. Only respond with a new list of [start_time, duration, sentence, float_score]. DO NOT give any answer outisde of the format. \n\"{input_file.read()}\""}] 
    }
    response = requests.post(url=url, headers=headers, json=data)
    sentiment = response.json().get('choices')[0].get('message').get('content')
    with open("sentiment_analysis.txt","w", encoding="utf-8") as sentanal: # hehe
        sentanal.write(sentiment)

Create Video from choosen segments

In [30]:
# Get the audio stream
video_stream = pytube.YouTube(vid_url).streams.filter().first()
# Check if audio stream is not None before attempting to download it
if video_stream is not None:
    # Download the audio stream
    video_file = video_stream.download(filename="input_video.mp4").replace("\\","/")
else:
    print("No audio stream found.")

newpath = os.getcwd() + "/videos/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

with open("sentiment_analysis.txt","r", encoding="utf-8") as sentanal: # again hehe
    for i, sentlist in enumerate(ast.literal_eval(sentanal.read())):
        if(float(sentlist[3]) >= sentiment_score):
            output_file_path = f"videos/output_{i}.mp4"
            split_command = ["ffmpeg", "-i", "input_video.mp4", "-ss", str(sentlist[0]), "-t", str(sentlist[1]), "-c", "copy", output_file_path]
            subprocess.run(split_command, check=True)

0

Combine the videos

In [32]:
# List of video files to combine
video_files = os.listdir('videos')
# Create a list of commands to concatenate the videos
commands = ['ffmpeg', '-i', f'concat:{"|".join(video_files_with_path)}', '-c', 'copy', 'output.mp4']

# Run the ffmpeg command
subprocess.call(commands)

Delete directories

In [31]:
if(delete_directories):
    newpath = ["/metadata/","/downloads/","/chunks/","/videos/"]
    for i in newpath:
        if os.path.exists(os.getcwd() + i):
            shutil.rmtree(os.getcwd() + i)

    newfilepath = ["/temporary.txt", "/video_transcription.txt","/sentiment_analysis.txt","/input.mp4","/input_video.mp4"]
    for i in newfilepath:
        if os.path.exists(os.getcwd() + i):
            os.remove(os.getcwd() + i)